In [1]:
from operator import itemgetter
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable.utils import ConfigurableField

In [2]:
load_dotenv()

True

In [16]:
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from")
prompt2 = ChatPromptTemplate.from_template(
    "What country is the city {city} in? respond in {language}"
)

model = ChatOpenAI(model = 'babbage-002').configurable_fields(
    temperature = ConfigurableField(
        id = "llm_temperature",
        name = "LLM Temperature",
        description = "The temperature of the LLM model",
    )
)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {'city': chain1, 'language': itemgetter('language')}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.with_config({'llm_temperature': 0.5}) \
    .invoke({'person': 'obama', 'language': 'spanish'})

NotFoundError: Error code: 404 - {'error': {'message': 'This is not a chat model and thus not supported in the v1/chat/completions endpoint. Did you mean to use v1/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [15]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda


def print_and_return(value, label):
    print(f"{label}: {value}")
    return value

print_color = RunnableLambda(lambda x: print_and_return(x, "Color"))
print_fruit = RunnableLambda(lambda x: print_and_return(x, "Fruit"))
print_country = RunnableLambda(lambda x: print_and_return(x, "Country"))


prompt1 = ChatPromptTemplate.from_template(
    "Generate a {attribute} color. Return the name of the color and nothing else:"
)

prompt2 = ChatPromptTemplate.from_template(
    "What is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)

prompt3 = ChatPromptTemplate.from_template(
    "What is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)

prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 |  {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser | print_fruit
color_to_country = prompt3 | model_parser | print_country
question_generator = (
    color_generator | {"fruit" : color_to_fruit, "country": color_to_country} | prompt4
)

total_answers = question_generator | model_parser

print(total_answers.invoke({"attribute": "warm"}))

Fruit: Coral
Country: Comoros
The color coral is typically described as a shade of pink or orange, resembling the color of coral reefs. 

The flag of Comoros consists of four horizontal stripes of yellow, white, red, and blue. The top stripe is yellow, followed by a white stripe, then a red stripe, and finally a blue stripe at the bottom. In the upper hoist-side corner, there is a green triangle with a white crescent and four white stars inside it.


In [19]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Gnerate a final response given the critique")
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)


chain = (
    planner 
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response")
    } 
    | final_responder
)

print(chain.invoke({"input": "scrum"}))

AttributeError: 'str' object has no attribute 'content'